In [2]:
!pip install kaggle

### Setup Kaggle Credentials:

1. Go to https://www.kaggle.com/
2. Sign in or create an account
3. Go to Account Settings (click your profile picture → Account)
4. Scroll to "API" section
5. Click "Create New API Token"
6. This downloads `kaggle.json` file
7. Upload it to this notebook or your environment

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

KeyboardInterrupt: 

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

!unzip -q chest-xray-pneumonia.zip -d ./dataset
print("✅ Dataset downloaded and extracted!")

### Method 2: Manual Download

If Kaggle API doesn't work:

1. Visit: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
2. Click "Download" button
3. Extract the zip file
4. Upload to your notebook environment

Dataset structure:
```
chest_xray/
├── train/
│   ├── NORMAL/
│   └── PNEUMONIA/
├── test/
│   ├── NORMAL/
│   └── PNEUMONIA/
└── val/
    ├── NORMAL/
    └── PNEUMONIA/
```

In [4]:
!pip install numpy pandas opencv-python scikit-image PyWavelets Pillow matplotlib seaborn scikit-learn xgboost ipywidgets
print("✅ All packages installed!")

^C
✅ All packages installed!


In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# Image processing
import cv2
from PIL import Image
from skimage.feature import graycomatrix, graycoprops
import pywt

# Visualization
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set_style("whitegrid")

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from xgboost import XGBClassifier

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output, HTML

import time
import pickle

print("✅ All libraries imported successfully!")